# Título   Crear clase ternaria
Vn. usada para Kaggle3

## Autor: Jose Chelquer
## Fecha de última modificación: 26/11/2024

Crear clase_ternaria a partir del crudo grabando otro dataset en formato parquet


## Parámetros

< Descripción de cada uno de los parámetros que utiliza el job >


In [1]:

usar_gdrive=False    # Poner en true si se va a ejecutar con archivos de google drive

## Input

< Archivos de datos (csv.gz) con sus paths que van a consumirse por el job>

In [2]:
dataset_path = '~/buckets/b1/datasets'
dataset_file = 'competencia_03_crudo.csv.gz'

## Output

< Archivos, bases de datos, modelos que va a generar el job>

In [3]:
# si no se quiere usar parquet.gz hay que cambiar la parte de grabación
output_file='k3_____.parquet.gz'


In [4]:
## Procesos

### Paquetes necesarios

## Código del proceso

< Todo el código a partir de aquí debe poder ejecutarse sin necesidad de parametrizar nada>

In [11]:
import os
import pandas as pd
import numpy as np

In [6]:
if usar_gdrive:
    from google.colab import drive
    drive.mount('/content/drive')

In [7]:
# leer el dataset

file_path = os.path.join(dataset_path, dataset_file)
print (f'Leyendo {file_path}')
# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)
df.shape

Leyendo ~/buckets/b1/datasets/competencia_03_crudo.csv.gz


(4901237, 154)

In [8]:
# Dado un mes en formato aaaamm y una cantidad de meses
# devuelve el mes a distancia meses de mes
def sig_mes(mes, meses):
  aaaa=mes//100
  mm=mes%100+meses
  aExtra=(mm-1)//12
  mm=mm-12*aExtra
  return (aaaa+aExtra)*100+mm


In [13]:
# Convertir a una lista de tuplas
meses=np.sort(df['foto_mes'].unique())
for mes in meses:
    print (mes, sig_mes(mes,1), sig_mes(mes, 2))


201901 201902 201903
201902 201903 201904
201903 201904 201905
201904 201905 201906
201905 201906 201907
201906 201907 201908
201907 201908 201909
201908 201909 201910
201909 201910 201911
201910 201911 201912
201911 201912 202001
201912 202001 202002
202001 202002 202003
202002 202003 202004
202003 202004 202005
202004 202005 202006
202005 202006 202007
202006 202007 202008
202007 202008 202009
202008 202009 202010
202009 202010 202011
202010 202011 202012
202011 202012 202101
202012 202101 202102
202101 202102 202103
202102 202103 202104
202103 202104 202105
202104 202105 202106
202105 202106 202107
202106 202107 202108
202107 202108 202109
202108 202109 202110
202109 202110 202111


In [14]:
df["cli_mes"]=df["numero_de_cliente"].astype(str)+df["foto_mes"].astype(str)
df["cli_mesMas1"]=df["numero_de_cliente"].astype(str)+df["foto_mes"].apply(lambda x: sig_mes(x, 1)).astype(str)
df["cli_mesMas2"]=df["numero_de_cliente"].astype(str)+df["foto_mes"].apply(lambda x: sig_mes(x, 2)).astype(str)

In [15]:
print (f'Detectando existe1 y existe2')
df1=df['cli_mes']   #sólo las columnas del cliente
# Realiza la unión entre df y df1
resultado = df.merge(df1, left_on='cli_mesMas1', right_on='cli_mes', how='left', indicator=True)
df['existe1'] = resultado['_merge'] == 'both'
resultado = df.merge(df1, left_on='cli_mesMas2', right_on='cli_mes', how='left', indicator=True)
df['existe2'] = resultado['_merge'] == 'both'


Detectando existe1 y existe2


In [16]:
print (f'Creando clase ternaria')
df[ "clase_ternaria"]="CONTINUA"          #default
df.loc[df["existe2"]==False, "clase_ternaria"]="BAJA+2"
df.loc[df["existe1"]==False, "clase_ternaria"]="BAJA+1"
df.loc[df["existe2"]==True, "clase_ternaria"]="CONTINUA"

Creando clase ternaria


In [17]:
# Eliminar los campos creados
print (f'Eliminando campos creados')
df = df.drop(columns=['cli_mes', 'cli_mesMas1', 'cli_mesMas2', 'existe1', 'existe2'])

Eliminando campos creados


In [18]:
df['clase_ternaria'].value_counts()

clase_ternaria
CONTINUA    4528247
BAJA+1       187609
BAJA+2       185381
Name: count, dtype: int64

In [19]:
df.shape[0]

4901237

In [20]:
ruta_nuevo_archivo = os.path.join(dataset_path, output_file)
# Guarda el DataFrame en el nuevo archivo
print (f'Guardando el archivo de salida')
df.to_parquet(ruta_nuevo_archivo, index=False, compression='gzip')
print(f"El archivo ha sido guardado en: {ruta_nuevo_archivo}")

Guardando el archivo de salida
El archivo ha sido guardado en: ~/buckets/b1/datasets/k3_____.parquet.gz
